In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


배수펌프 = pd.read_csv('./데이터/Green/서울시 배수펌프장 정보.csv',encoding='cp949')
녹지 = pd.read_csv('./데이터/Green/서울시_녹지현황.csv',encoding='utf-8')
습윤지수 = pd.read_csv('./데이터/Green/서울시_습윤지수_병합.csv',encoding='cp949')
불투수율 = pd.read_csv('./데이터/Green/서울특별시_자치구별 불투수면적 현황.csv',encoding='cp949')
침수흔적도 = pd.read_csv('./데이터/서울시_침수흔적도.csv',encoding='utf-8')
빗물펌프거리= pd.read_csv('./데이터/서울시_빗물펌프거리.csv',encoding='utf-8')
하천거리 = pd.read_csv('./데이터/서울시_빗물펌프거리.csv',encoding='utf-8')

# 배수펌프
- 시설 관리자 구형식 : 시설관리자 => 지역구로 변경
- 표고 관련 column : null값 너무 많아서 뺌
- 전양정,실양정 : null값 너무 많아서 뺌
전양정,실양정 관련 글 : https://blog.naver.com/PostView.nhn?blogId=hvackkw&logNo=220875366802
- 지역구 : 서울시 => 구로구 인거 확인, 나머지도 %구 형식으로 변경

In [15]:
# 시설관리자가 구형식으로 되어있어서 null값에 구단위로 추가 후 구단위로 변경
배수펌프.loc[배수펌프['시설관리자'].isnull(), '시설관리자'] = 배수펌프['상세주소'].str.extract(r'(\S+구)')[0]

배수펌프 = 배수펌프.rename(columns = {'시설관리자': '자치구'})

In [16]:
# 자치구 조정 함수
def adjust_district_names(df):
    # '서울시'를 '구로구'로 변경
    df['자치구'] = df['자치구'].replace('서울시', '구로구')
    
    # '구청'을 '구'로 변경
    df['자치구'] = df['자치구'].str.replace('구청', '구')

    # '구구'가 되는 경우를 '구'로 변경
    df['자치구'] = df['자치구'].str.replace('구구', '구')
    
    # 모든 지역구가 '구'로 끝나도록 조정
    df['자치구'] = df['자치구'].apply(lambda x: x if x.endswith('구') else x.split()[0] + '구')
    
    return df

# 조정된 지역구 데이터프레임
배수펌프 = adjust_district_names(배수펌프)

In [17]:
배수펌프['상세주소']

0      서울시 구로구 구로4동 120-5
1                 풍납동 471
2               방이동 88-13
3               방이동 88-13
4               방이동 88-13
             ...         
91       서울시 성동구 송정동 78-1
92      서울시 동대문구 송정동 78-1
93       서울시 동대문구 장안동 356
94    서울시 동대문구 장안4동 301-4
95      서울시 동대문구 장안1동 379
Name: 상세주소, Length: 96, dtype: object

In [18]:
배수펌프.isna().sum()

하천기본계획코드       0
일련번호           0
하천기본계획 사업명     0
수립년도           0
시설물명           0
하천명            0
하천등급           0
상세주소           0
주소            68
유역면적           1
보호면적          10
시행기관          86
본류하천_유역면적      5
본류하천_홍수량       9
본류하천_계획빈도      9
제방고            9
계획홍수위          3
부지표고          61
기계실바닥표고       62
전기실바닥표고       87
배수장_최대배수량      1
배수장_계획빈도       1
초기흡입수위        24
전동기            0
펌프규격          29
전양정           58
실양정           91
토출수위          41
자치구            0
구분            22
배수문규격         33
배수문바닥표고       42
간선수로          89
지선수로          89
유수지표고_HWL     95
유수지표고_LWL     86
유수지면적         61
유수지용량         42
dtype: int64

In [19]:
배수펌프 = 배수펌프[['자치구','상세주소','시설물명','유역면적','보호면적','본류하천_유역면적','본류하천_홍수량','제방고','계획홍수위','배수장_최대배수량','초기흡입수위','펌프규격','토출수위','배수문규격','유수지면적','유수지용량']]

KeyError: "['지역구'] not in index"

# 녹지
- 세부지역 => 지역구로 변경
- 소계 => 사용 x  삭제
- 구단위 => 중부공원 녹지사업소, 안전총괄실=> 중구로 합치기

In [ ]:
# '소계'인 행을 제거
녹지 = 녹지[녹지['세부지역'] != '소계']


# NaN 값을 0으로 대체
녹지.fillna(0, inplace=True)
# '중구' 행을 찾기
중구_행 = 녹지[녹지['세부지역'] == '중구']

# '중부공원녹지사업소'와 '안전총괄실' 행을 찾기
중부공원_녹지사업소_행 = 녹지[녹지['세부지역'] == '중부공원녹지사업소']
안전총괄실_행 = 녹지[녹지['세부지역'] == '안전총괄실']

# 수치 데이터 컬럼만 선택하여 중구 행에 합산
수치_컬럼 = 녹지.columns[1:]  # 나머지 컬럼도 추가
중구_행[수치_컬럼] += 중부공원_녹지사업소_행[수치_컬럼].iloc[0] + 안전총괄실_행[수치_컬럼].iloc[0]

# '중부공원녹지사업소'와 '안전총괄실' 행을 데이터프레임에서 제거
녹지 = 녹지[~녹지['세부지역'].isin(['중부공원녹지사업소', '안전총괄실'])]

# 변경된 '중구' 행을 원본 데이터프레임에 업데이트
녹지.loc[녹지['세부지역'] == '중구', 수치_컬럼] = 중구_행[수치_컬럼]

녹지['세부지역'] = 녹지['세부지역'].replace('중구중부공원녹지사업소안전총괄실', '중구')

녹지 = 녹지.rename(columns = {'세부지역': '자치구'})


# '지역'컬럼 삭제
녹지= 녹지.drop(columns=['지역'])

C:\Users\ciw96\AppData\Local\Temp\ipykernel_42124\581113263.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  중구_행[수치_컬럼] += 중부공원_녹지사업소_행[수치_컬럼].iloc[0] + 안전총괄실_행[수치_컬럼].iloc[0]
C:\Users\ciw96\AppData\Local\Temp\ipykernel_42124\581113263.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[414.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  녹지.loc[녹지['세부지역'] == '중구', 수치_컬럼] = 중구_행[수치_컬럼]
C:\Users\ciw96\AppData\Local\Temp\ipykernel_42124\581113263.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1944645.0]' has dtype incompatible

In [ ]:
녹지.columns

Index(['지역구', '총개소수', '총면적', '시설녹지_개소수', '시설녹지_면적', '일반녹지_개소수', '일반녹지_면적',
       '중앙분리대_개소수', '중앙분리대_면적', '교통섬_개소수', '교통섬_면적', '화단_개소수', '화단_면적',
       '보도부_개소수', '보도부_면적', '기타녹지_개소수', '기타녹지_면적', '지하철환기구_개소수', '지하철환기구_면적',
       '건물주변_개소수', '건물주변_면적', '아파트학교_개소수', '아파트학교_면적', '침수공간_개소수', '침수공간_면적',
       '기타_개소수', '기타_면적', '녹지율'],
      dtype='object')

In [ ]:
습윤지수 = 습윤지수.rename(columns={'구별코드' :'자치구'})

In [ ]:
습윤지수

,지역구,TWI_min,TWI_max,TWI_count,TWI_counts
0,강남구,0.956859,22.990770,256,10|21|67|148|218|374|576|934|1193|1619|1997|25...
1,강동구,0.795508,26.145687,256,1|3|4|13|28|75|140|226|405|613|879|1221|1653|1...
2,강북구,0.442591,22.733292,256,1|3|19|66|119|252|465|743|1279|1843|2439|3192|...
3,강서구,0.588825,26.614878,256,5|3|30|41|78|191|372|644|934|1214|1483|1816|20...
4,관악구,0.451903,23.116009,256,3|9|41|107|178|362|626|1084|1671|2653|3515|439...
5,광진구,0.631392,20.507568,256,4|9|9|34|49|89|155|302|381|514|587|744|898|949...
6,구로구,0.939221,22.691904,256,6|7|28|48|109|158|319|517|737|948|1363|1734|19...
7,금천구,0.605828,24.525225,256,5|5|10|20|43|70|163|364|511|755|986|1386|1679|...
8,노원구,0.615565,25.458551,256,5|16|23|85|171|324|591|994|1542|2257|2912|3590...
9,도봉구,0.483903,24.166096,256,6|14|34|56|95|144|180|296|433|587|755|1134|133...


# 불투수율 

- 비고 삭제

In [ ]:
불투수율.drop(columns=['비고'])

,연번,자치구,구 전체면적(헥타르),불투수면적(헥타르),불투수면적 비율(퍼센트)
0,1,종로구,2312.4,971.6,42.02
1,2,중구,998.2,749.2,75.05
2,3,용산구,2196.7,970.0,44.16
3,4,성동구,1642.2,1040.4,63.36
4,5,광진구,1788.9,1023.9,57.23
5,6,동대문구,1459.4,1078.3,73.89
6,7,중랑구,1829.9,1070.6,58.51
7,8,성북구,2487.1,1305.1,52.48
8,9,강북구,2381.9,861.2,36.15
9,10,도봉구,2077.7,839.8,40.42


# 침수흔적도 

- 침수 시작 날짜 : 218로 시작=> 2018로 바꿈
- 침수 종료 날짜 : 틀 맞지 않는 형식 %Y%m%d형식으로 변경
-          추가  : Nan값들 침수시작날짜와 동일하게 변경

In [ ]:
침수흔적도 = 침수흔적도[(침수흔적도['침수시작날짜']>=20180101) & (침수흔적도['침수시작날짜']<=20231231)]
침수흔적도

In [ ]:
#
침수흔적도['침수시작날짜'] = 침수흔적도['침수시작날짜'].replace(2180828, 20180828)

# '침수종료날짜' 컬럼의 날짜 형식 통일
침수흔적도['침수종료날짜'] = pd.to_datetime(침수흔적도['침수종료날짜'], errors='coerce').dt.strftime('%Y%m%d')


# 'NaN'을 실제 NaN 값으로 변환
침수흔적도['침수종료날짜'] = 침수흔적도['침수종료날짜'].replace('NaN', np.nan)

# '침수종료날짜'의 NaN 값을 '침수시작날짜'로 대체
침수흔적도['침수종료날짜'] = pd.to_datetime(침수흔적도['침수종료날짜'], errors='coerce').dt.strftime('%Y%m%d')
침수흔적도['침수종료날짜'] = 침수흔적도['침수종료날짜'].fillna(침수흔적도['침수시작날짜'].astype(str))


In [ ]:
# 침수날짜 컬럼 생성: 각 행에 대해 침수 시작부터 종료까지의 모든 날짜 생성
침수흔적도['침수날짜'] = 침수흔적도.apply(
    lambda x: pd.date_range(
        start=pd.to_datetime(str(x['침수시작날짜']), format='%Y%m%d'),  
        end=pd.to_datetime(str(x['침수종료날짜']), format='%Y%m%d'),    
        freq='D'
    ).strftime('%Y%m%d').tolist(),
    axis=1
)

# 기존 '침수시작날짜', '침수종료날짜' 컬럼 제거
침수흔적도.drop(['침수시작날짜', '침수종료날짜'], axis=1, inplace=True)

# explode를 사용하여 '침수날짜'의 각 날짜를 독립적인 행으로 확장
침수흔적도 = 침수흔적도.explode('침수날짜')

print(침수흔적도)



       지역구  침수수심                  침수원인  침수평균해발고도     침수면적        행정코드  침수시작시간  \
0      강북구  0.02     2022년 8.8 ~17. 호우    114.53   3876.0  1130510400      18   
1      강북구  0.02     2022년 8.8 ~17. 호우    118.05   2188.0  1130510400      18   
2      강북구  0.02     2022년 8.8 ~17. 호우    129.55  45287.0  1130510400      18   
3     영등포구  0.30     2022년 8.8 ~17. 호우     10.18     53.0  1156010400      20   
3     영등포구  0.30     2022년 8.8 ~17. 호우     10.18     53.0  1156010400      20   
...    ...   ...                   ...       ...      ...         ...     ...   
9723   서초구  0.50  2016년 7.1~7 호우·강풍·풍랑     18.47    281.9  1165010200       5   
9723   서초구  0.50  2016년 7.1~7 호우·강풍·풍랑     18.47    281.9  1165010200       5   
9723   서초구  0.50  2016년 7.1~7 호우·강풍·풍랑     18.47    281.9  1165010200       5   
9723   서초구  0.50  2016년 7.1~7 호우·강풍·풍랑     18.47    281.9  1165010200       5   
9723   서초구  0.50  2016년 7.1~7 호우·강풍·풍랑     18.47    281.9  1165010200       5   

      침수종료시간         침수내용  

In [ ]:
침수흔적도

,지역구,침수수심,침수원인,침수평균해발고도,침수면적,행정코드,침수시작시간,침수종료시간,침수내용,주소,침수연도,침수지역정보,지오메트리,침수날짜
0,강북구,0.02,2022년 8.8 ~17. 호우,114.53,3876.0,1130510400,18,23,내수침수,서울특별시 강북구 우이동 296-1,2022,주택,"POLYGON ((956425.1271956386 1963792.043624141,...",20220808
1,강북구,0.02,2022년 8.8 ~17. 호우,118.05,2188.0,1130510400,18,23,내수침수,서울특별시 강북구 우이동 299-2,2022,주택,"POLYGON ((956590.6408143371 1963700.11134354, ...",20220808
2,강북구,0.02,2022년 8.8 ~17. 호우,129.55,45287.0,1130510400,18,23,내수침수,서울특별시 강북구 우이동 산8-15,2022,주택,"POLYGON ((956383.2795007367 1963392.863235048,...",20220808
3,영등포구,0.30,2022년 8.8 ~17. 호우,10.18,53.0,1156010400,20,24,내수침수,서울특별시 영등포구 영등포동3가 13-7,2022,상가,"POLYGON ((947842.214000747 1946702.945905377, ...",20220808
3,영등포구,0.30,2022년 8.8 ~17. 호우,10.18,53.0,1156010400,20,24,내수침수,서울특별시 영등포구 영등포동3가 13-7,2022,상가,"POLYGON ((947842.214000747 1946702.945905377, ...",20220809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9723,서초구,0.50,2016년 7.1~7 호우·강풍·풍랑,18.47,281.9,1165010200,5,11,침수(공공하수도역류),서초구 양재동 322-3,2016,NaN,"POLYGON ((959412.3280236096 1941367.591284027,...",20160703
9723,서초구,0.50,2016년 7.1~7 호우·강풍·풍랑,18.47,281.9,1165010200,5,11,침수(공공하수도역류),서초구 양재동 322-3,2016,NaN,"POLYGON ((959412.3280236096 1941367.591284027,...",20160704
9723,서초구,0.50,2016년 7.1~7 호우·강풍·풍랑,18.47,281.9,1165010200,5,11,침수(공공하수도역류),서초구 양재동 322-3,2016,NaN,"POLYGON ((959412.3280236096 1941367.591284027,...",20160705
9723,서초구,0.50,2016년 7.1~7 호우·강풍·풍랑,18.47,281.9,1165010200,5,11,침수(공공하수도역류),서초구 양재동 322-3,2016,NaN,"POLYGON ((959412.3280236096 1941367.591284027,...",20160706


In [ ]:
빗물펌프거리  = 빗물펌프거리.rename(columns={'구단위':'자치구'})

In [ ]:
하천거리 = 하천거리.rename(columns={'구단위' : '자치구'})

In [ ]:
침수흔적도['지오메트리'][0]

'POLYGON ((956425.1271956386 1963792.043624141, 956424.8854296223 1963792.446760627, 956421.6884048309 1963796.543246887, 956418.2632162417 1963800.405033851, 956416.8117928583 1963801.728263483, 956416.4236773055 1963801.303485454, 956415.673762397 1963802.019208943, 956413.8817620513 1963803.398229473, 956420.995892275 1963802.790613666, 956421.7707828018 1963809.404039349, 956425.7187906543 1963815.696705643, 956425.8033045963 1963815.799218082, 956434.6548788747 1963826.488171006, 956435.3412276129 1963823.593593678, 956436.8860344886 1963786.817015585, 956437.7349660392 1963768.627245897, 956495.3452938531 1963810.305350579, 956524.1096722074 1963776.098037665, 956516.0908221444 1963772.371073411, 956522.9037674818 1963753.847705869, 956517.5043732296 1963749.449057447, 956515.3938606675 1963740.933442291, 956502.5895697867 1963742.429994271, 956487.0221593921 1963745.967486548, 956462.8295554036 1963753.56735047, 956460.856231007 1963766.90490005, 956433.1606194369 1963759.031136

In [ ]:
from functools import reduce

dfs = [배수펌프, 녹지,습윤지수,불투수율,침수흔적도]

df = reduce(lambda left, right:pd.merge(left,right, on='자치구',how='outer'),dfs)


df

KeyError: '자치구'

In [ ]:
배수펌프

,지역구,상세주소,시설물명,유역면적,보호면적,본류하천_유역면적,본류하천_홍수량,제방고,계획홍수위,배수장_최대배수량,초기흡입수위,펌프규격,토출수위,배수문규격,유수지면적,유수지용량
0,구로구,서울시 구로구 구로4동 120-5,구로2 빗물펌프장,0.5,"382,000",42.5,549,16.46,14.85,21.9,6.8,"1,100×3, 700×1, 1650×2",8.5,3.00×2.00@2,962,"1,904"
1,송파구,풍납동 471,몽촌2,8.2,"580,000",29.6,282,22.13,14.00,21.1,10.6,"1,400×4, 700×2",14.0,4.0×4.0×10,NaN,NaN
2,강동구,방이동 88-13,성내,6.3,"1,130,000",29.6,282,17.72,14.31,75,10.0,"1,500×10, 1,800×3",13.0,4.0×3.0×2,"59,508","237,000"
3,강동구,방이동 88-13,성내,6.3,"1,130,000",29.6,282,17.72,14.31,75,10.0,"1,500×10, 1,800×3",13.0,2.5×1.5×1,NaN,NaN
4,강동구,방이동 88-13,성내,6.3,"1,130,000",29.6,282,17.72,14.31,75,10.0,"1,500×10, 1,800×3",13.0,6.0×3.0×2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,성동구,서울시 성동구 송정동 78-1,"전농1,2",6.2,"620,000",239.6,"2,040",18.89,17.05,"3,395",7.6,NaN,NaN,NaN,"20,500","102,000"
92,동대문구,서울시 동대문구 송정동 78-1,장안,2.8,"480,000",239.6,"2,040",17.87,17.05,"1,350",7.6,NaN,NaN,NaN,"17,400","70,000"
93,동대문구,서울시 동대문구 장안동 356,장안2,0.2,"200,000",239.6,"2,040",18.92,17.05,120,NaN,"600, 400",NaN,NaN,NaN,600
94,동대문구,서울시 동대문구 장안4동 301-4,장안4 (구장안3),1.2,"1,150,000",239.6,"2,040",18.80,17.05,650,NaN,NaN,NaN,NaN,NaN,"3,250"


In [ ]:
배수펌프['지역구'].unique()

array(['구로구', '송파구', '강동구', '강남구', '서초구', '금천구', '관악구', '영등포구', '마포구',
       '은평구', '성동구', '광진구', '중랑구', '동대문구', '서구', '노원구', '성북구'],
      dtype=object)